In [ ]:
import numpy as np
import cv2

net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
# Load the COCO dataset 
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Random colors for each class label
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Start
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        img = frame
        height, width, n_channels = img.shape
        blob = cv2.dnn.blobFromImage(img, 0.0032, (256, 256), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        class_ids = []
        boxes = []
        confidences = []

        # Loop through all output
        for out in outs:
            for det in out:
                scores = det[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.6:
                    cx = int(det[0] * width)
                    cy = int(det[1] * height)
                    w = int(det[2] * width)
                    h = int(det[3] * height)

                    x = int(cx - w / 2)
                    y = int(cy - h / 2)

                    # Adding bounding box, confidences, and class IDs
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Seting font type, size, and thickness
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        font_thickness = 3

        # Loop through detected objects and draw bounding boxes
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[class_ids[i]]
                conf = confidences[i]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, f"{label} {round(conf, 2)}", (x, y - 10), font, font_scale, color, font_thickness)

        # Showing the resulting frame
        cv2.imshow('YOLO Object Detection', img)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

#capture and close window
cap.release()
cv2.destroyAllWindows()


2024-11-07 14:02:12.282 python[6528:640184] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-07 14:02:12.282 python[6528:640184] +[IMKInputSession subclass]: chose IMKInputSession_Modern
